In [ ]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
df = pd.read_csv(r'datasets\electricity_data.csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df[df['Date'].dt.year == 2021]
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head(3)

In [ ]:
df.shape

In [ ]:
import math
import statsmodels.api as sm
import statsmodels.tsa.api as smt
from sklearn.metrics import mean_squared_error
from matplotlib import pyplot
import matplotlib.pyplot as plt

actual_vals = df.Total_Consumption.values
actual_log = np.log10(actual_vals)

train, test = actual_vals[0:-80], actual_vals[-80:]
train_log, test_log = np.log10(train), np.log10(test)
my_order = (1, 1, 1)
my_seasonal_order = (1, 1, 1, 7)

In [ ]:
history = [x for x in train_log]
predictions = list()
predict_log=list()
for t in range(len(test_log)):
    model = sm.tsa.SARIMAX(history, order=my_order, seasonal_order=my_seasonal_order,enforce_stationarity=False,enforce_invertibility=False)
    model_fit = model.fit(disp=0)
    output = model_fit.forecast()
    predict_log.append(output[0])
    yhat = 10**output[0]
    predictions.append(yhat)
    obs = test_log[t]
    history.append(obs)
   # print('predicted=%f, expected=%f' % (output[0], obs))
#error = math.sqrt(mean_squared_error(test_log, predict_log))
#print('Test rmse: %.3f' % error)
# plot
figsize=(12, 7)
plt.figure(figsize=figsize)
pyplot.plot(test,label='Actuals')
pyplot.plot(predictions, color='red',label='Predicted')
pyplot.legend(loc='upper right')
pyplot.show()

In [ ]:
df_preds = df[-80:]
df_preds['Predicted_Consumption'] = predictions

In [ ]:
import plotly.graph_objects as go
import plotly.offline as py

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_preds['Date'], y=df_preds['Total_Consumption'],
                    mode='lines',
                    name='lines'))
fig.add_trace(go.Scatter(x=df_preds['Date'], y=df_preds['Predicted_Consumption'],
                    mode='lines',
                    name='lines'))

In [ ]:
new_dates = pd.date_range(start='2021/07/31', end='2021/09/30')

In [ ]:
train = df.Total_Consumption.values

my_order = (1, 1, 1)
my_seasonal_order = (1, 1, 1, 62)

model = sm.tsa.SARIMAX(train, order=my_order, seasonal_order=my_seasonal_order,enforce_stationarity=False,enforce_invertibility=False)
model_fit = model.fit()
output = model_fit.forecast(62)

In [ ]:
listofzeros = [np.NaN] * 62

In [ ]:
data_new = {'Date':new_dates,
            'Total_Consumption':listofzeros,
            'Predicted_Consumption':output}

df_new_preds = pd.DataFrame(data_new)

In [ ]:
df_all = pd.concat([df_preds, df_new_preds], ignore_index=True)

In [ ]:
df_all.head(2)

In [ ]:
df_all['MAE'] = df_all['Total_Consumption'] - df_all['Predicted_Consumption']

In [ ]:
df_anoms = df_all[df_all['MAE'] >= 15]
df_anoms.reset_index(drop=True, inplace=True)

In [ ]:
df_anoms.head(2)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_all['Date'], y=df_all['Total_Consumption'],
                    mode='lines',
                    name='Actual Consumption'))
fig.add_trace(go.Scatter(x=df_all['Date'], y=df_all['Predicted_Consumption'],
                    mode='lines',
                    name='Forecasted Consumption'))
fig.add_trace(go.Scatter(x=df_anoms['Date'], y=df_anoms['Total_Consumption'],
                    mode='markers',
                    name='Excess Consumption'))
fig.update_traces(marker=dict(size=5, 
                              line=dict(width=5,
                                        color='red')))

In [ ]:
import smtplib, ssl
from smtplib import SMTPException
from pathlib import Path
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email.mime.text import MIMEText
from email.utils import COMMASPACE, formatdate
from email import encoders
import plotly
import datetime

for i in df_anoms['Date']:
    fig = go.Figure()
    df_anoms_temp = df_anoms[df_anoms['Date'] == i]
    df_anoms_temp.reset_index(drop=True, inplace=True)
    df_all_temp = df_all[df_all['Date'] <= i + datetime.timedelta(days=1)]
    df_all_temp.reset_index(drop=True, inplace=True)
    fig.add_trace(go.Scatter(x=df_all_temp['Date'], y=df_all_temp['Total_Consumption'],
                        mode='lines',
                        name='Actual Consumption'))
    fig.add_trace(go.Scatter(x=df_all_temp['Date'], y=df_all_temp['Predicted_Consumption'],
                        mode='lines',
                        name='Forecasted Consumption'))
    fig.add_trace(go.Scatter(x=df_anoms_temp['Date'], y=df_anoms_temp['Total_Consumption'],
                        mode='markers',
                        name='Excess Consumption'))
    fig.update_traces(marker=dict(size=5, 
                                  line=dict(width=5,
                                            color='red')))
    i = str(i)
    i = i[:-9]
    plotly.io.write_image(fig, 'output_file'+i+'.pdf', format='pdf')
    SUBJECT = "Early Warning - Abnormal usage of Electricity Detected! - Dated:" + i

    msg = MIMEMultipart()
    msg['Subject'] = SUBJECT 
    msg['From'] = 'debadriworkshop@gmail.com'
    msg['To'] = 'debadridtt@gmail.com'

    body = "Electra.AI has detected unusal usage of electricity. This is an early warning generated to keep the electricity bill under control."

    msg.attach(MIMEText(body, 'plain'))

    filename = 'output_file'+i+'.pdf'
    attachment = open('output_file'+i+'.pdf', "rb")

    part = MIMEBase('application', "octet-stream")
    part.set_payload((attachment).read())
    encoders.encode_base64(part)

    part.add_header('Content-Disposition', "attachment; filename= %s" % filename)

    msg.attach(part)
    context = ssl.create_default_context()
    server = smtplib.SMTP('smtp.gmail.com', 587)
    server.ehlo()
    server.starttls(context=context)
    server.ehlo()
    server.login(msg['From'], '********')
    server.sendmail(msg['From'], msg['To'], msg.as_string())
    print('Email sent successfully for Date:', i)